<a href="https://colab.research.google.com/github/betoborda/pucp-pry-analisis-datos/blob/main/dashboard_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np
import functools
import io

import IPython
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import datetime


from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# ipywidgets
import ipywidgets as widgets

from ipywidgets import Button, Label, Image
from ipywidgets import Box, VBox, HBox
from ipywidgets import AppLayout, Layout

In [67]:
%load_ext google.colab.data_table

from google.colab import data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [68]:
# Variables globales sobre el dataset

np.random.seed(42)

model = {}

num_columns = ['COSTO ALQUILER DIARIO', 'TOTAL ALQUILER', 'COSTO CONSERJE DIARIO', 
               'TOTAL CONSERJE' ,'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES',
               'LIQUIDACION EN DOLARES']

model["df_original"] = None
model["df_final"] = None
model["df"] = None
model["main_output"] = None
model["num_columns"] = num_columns
model["df_reglas"] = None

model['ciudad']=None
model['tipo']=None
model['categoria']=None
model['dia_in']=None
model['hora_in']=None
model['dia_out']=None
model['hora_out']=None
model['moneda']=None
model['alquiler_diario']=None
model['dias_conserje']=None
model['conserje_diario']=None
model['adicionales_soles']=None
model['agente']=None
model['estado_transaccion']=None

df_errores = widgets.Output()

In [69]:
###########
#. Errores (TODO)
###########

download_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

#Widgets para dataframe errores
# TODO renombar variable output
df_errores = widgets.Output()

#funcion graficar pie
def show_error_graph(pie_output):
  global model

  with pie_output:
    clear_output()
    dat = [model['df'].shape[0], model['df_final'].shape[0]]
    index=['Errores','Validos']
    plt.pie(dat, labels=index,autopct='%1.1f%%',)
    plt.title('CALIDAD DE LA DATA')
    plt.show()


#Funcionalidad N°8: Mostrar en el otro tab df los errores (esta dentro de la funcion reglas)
def show_errors(df):
    columns = ['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']
    with df_errores:
        clear_output()
        display(df[columns])


def download_file(d):
    print("download df.xlsx", df.shape)
    df.to_excel("df.xlsx")


# Subscribir eventos
download_btn.on_click(download_file)


calidad_items = [df_errores]
calidad_column = widgets.VBox(calidad_items)

error_tab = widgets.VBox(
    children=[
        widgets.Label("Errores destacados"),
        download_btn,
        calidad_column]
)

In [70]:
##############
# Carga Page
##############

def process_rules(checkboxes):
  global model,df1_final

  df = model['df_original'].copy()

  if checkboxes[0].value:
    df['valcostoalquiler_REGLA1'] = np.where(~np.isclose(df['COSTO ALQUILER DIARIO']*df['DIAS ALQUILER'],df['TOTAL ALQUILER']), 1, 0)
  else:
    df['valcostoalquiler_REGLA1'] = 0
    
  if checkboxes[1].value:
    df['valcostoconserje_REGLA2'] = np.where(~np.isclose(df['DIAS CONSERJE']*df['COSTO CONSERJE DIARIO'],df['TOTAL CONSERJE']), 1, 0)
  else:
    df['valcostoconserje_REGLA2'] = 0

  if checkboxes[2].value:
    df['valdiasalquiler_REGLA3'] = np.where(~np.isclose(df['DIAS ALQUILER'], (df['CHECK OUT'] - df['CHECK IN']).dt.ceil('d').dt.days), 1, 0)
  else:
    df['valdiasalquiler_REGLA3'] = 0

  if checkboxes[3].value:
    df['valcheckout_REGLA4'] = np.where((df['CHECK OUT'].isnull()) & (df['ESTADO TRANSACCION']!='ANULADO'), 1, 0)
  else:
    df['valcheckout_REGLA4'] = 0

  if checkboxes[4].value:
    df['valadicionalusd_REGLA5'] = np.where((df['MONEDA']=='USD') & (df['COSTOS ADICIONALES SOLES']!=0), 1, 0)
  else:
    df['valadicionalusd_REGLA5'] = 0

  if checkboxes[5].value:
    df['valsumacostosusd_REGLA6'] = np.where((df['MONEDA']=='USD') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valsumacostosusd_REGLA6'] = 0

  if checkboxes[6].value:
    df['valsumacostospen_REGLA7'] = np.where((df['MONEDA']=='PEN') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE']+df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valsumacostospen_REGLA7'] = 0

  if checkboxes[7].value:
    df['valpenusdpen_REGLA8'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valpenusdpen_REGLA8'] = 0

  if checkboxes[8].value:
    df['valpenusdusd_REGLA9'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER'] + df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valpenusdusd_REGLA9'] = 0
  
  df['errores'] = np.where((df['valcostoalquiler_REGLA1']==1) | 
                           (df['valcostoconserje_REGLA2']==1) | 
                           (df['valdiasalquiler_REGLA3']==1) | 
                           (df['valcheckout_REGLA4']==1) | 
                           (df['valadicionalusd_REGLA5']==1) | 
                           (df['valsumacostosusd_REGLA6']==1) | 
                           (df['valsumacostospen_REGLA7']==1) | 
                           (df['valpenusdpen_REGLA8']==1)| 
                           (df['valpenusdusd_REGLA9']==1), 1, 0)
  
   #### Añadir la columna Regla en el df(errores)
  conditions = [
    (df['valcostoalquiler_REGLA1'] ==1),
    (df['valcostoconserje_REGLA2']==1),
    (df['valdiasalquiler_REGLA3'] ==1),
    (df['valcheckout_REGLA4']==1),
    (df['valadicionalusd_REGLA5']==1),
    (df['valsumacostosusd_REGLA6']==1),
    (df['valsumacostospen_REGLA7']==1),
    (df['valpenusdpen_REGLA8']==1),
    (df['valpenusdusd_REGLA9']==1)]
  choices = ['Regla_1', 'Regla_2', 'Regla_3','Regla_4','Regla_5','Regla_6','Regla_7','Regla_8','Regla_9',]
  df['REGLA'] = np.select(conditions, choices)

  # TODO actualizar logica
  model['df_final'] = df[df['errores']==0].iloc[:,0:19] ### Df limpia
  model["df_reglas"]=df[df['errores']==1].iloc[:,0:30]

  df = df[df['errores']==1].iloc[:,0:19] #### Df con errores
  model['df'] = df

  df1_final=model['df_final'].copy()


def on_choose_file_change(change):
  pass


def on_upload_file_clicked(b, choose_file_btn, filename_txt):
  global model

  def leerArchivoCSV(botonCargar, nom_arch):
    return pd.read_excel(io.BytesIO(botonCargar.value[nom_arch]['content']))

  csv_nombre = next(iter(choose_file_btn.value))
  model["df_original"]=leerArchivoCSV(choose_file_btn, csv_nombre)
  filename_txt.value=csv_nombre


def on_process_file_clicked(b, rule_chk_list, pie_out):
  process_rules(rule_chk_list)
  #process_imputers()
  show_error_graph(pie_out)
  # TODO mostrar los errores del procesamiento en la otra pestaña
  #show_errors(df) 


def create_carga_buttons():
  choose_file_btn = widgets.FileUpload(
    accept='xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    icon = 'upload'
  )
  file_name_txt = widgets.Text(
    placeholder='Ningún archivo cargado',
    disabled=True
  )
  upload_file_btn = widgets.Button(
    description='subir archivo',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  process_file_btn = widgets.Button(
    description='Procesar',
    button_style='success')

  upload_file_btn.on_click(functools.partial(on_upload_file_clicked, 
                                             choose_file_btn=choose_file_btn,
                                             filename_txt=file_name_txt))
  
  return widgets.HBox([choose_file_btn, 
                      file_name_txt, 
                      upload_file_btn, 
                      process_file_btn])


# panel = pestaña
def create_carga_tab_panel():

  rule_names = ['R1: total alquiler = días alquiler * costo alquiler diario', 
                'R2: total conserje = días conserje * costo conserje diario', 
                'R3: días alquiler = check out - check in', 
                'R4: Si check out vacío => estado transacción = Anulado', 
                'R5: Si moneda USD => no hay costos adicionales soles', 
                'R6: Si moneda USD => Total alquiler + Total conserje == liquidación en dólares', 
                'R7: Si moneda PEN => Total alquiler + Total conserje + costos adicionales soles == liquidación soles', 
                'R8: Si moneda PEN/USD => costos adicionales soles == liquidación en soles', 
                'R9: Si moneda PEN/USD => Total alquiler + Total conserje == liquidación en dólares']

  def create_checkbox(description):
    #TODO crear un un checkbox, un label para la regla y agruparlos
    return widgets.Checkbox(value=True, description=description, indent=False)

  rule_checkboxes = [create_checkbox(description=rule_name) for rule_name in rule_names]

  consistencia_column = widgets.VBox(children=[widgets.Label("Consistencia"),
                                     *rule_checkboxes], 
                                     layout=Layout(width='auto'))

  limpieza_items = [widgets.Label("Limpieza"), 
                    widgets.Checkbox(description="Nulos", disabled=True) ]
  limpieza_column = widgets.VBox(limpieza_items)

  #Widgets para pie
  pie_output = widgets.Output()

  calidad_items = [widgets.Label("Calidad de los datos"), pie_output]
  calidad_column = widgets.VBox(calidad_items)

  carga_buttons_hbox = create_carga_buttons()
  process_file_btn = carga_buttons_hbox.children[3]
  process_file_btn.on_click(functools.partial(on_process_file_clicked,
                                              rule_chk_list=rule_checkboxes,
                                              pie_out=pie_output))

  return widgets.VBox(
    children=[
        carga_buttons_hbox,
        widgets.HBox(
            children=[consistencia_column, 
                      limpieza_column, 
                      calidad_column],
            layout=Layout(width='auto')
        )
    ]
  )

####################PESTAÑA ERRORES



def show_errors(df):
    #columns = ['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
     #          'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
     #          'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']
    with df_errores:
        clear_output()
        display(df)

def filter_error_rules(a):
    global df1
    with df_errores:
      clear_output()
      if dropdown_rule_btn.value=='Regla 1':
        df1=df[df['REGLA']=='Regla_1']
      elif dropdown_rule_btn.value=='Regla 2':
        df1=df[df['REGLA']=='Regla_2']
      elif dropdown_rule_btn.value=='Regla 3':
        df1=df[df['REGLA']=='Regla_3']
      elif dropdown_rule_btn.value=='Regla 4':
        df1=df[df['REGLA']=='Regla_4']
      elif dropdown_rule_btn.value=='Regla 5':
        df1=df[df['REGLA']=='Regla_5']
      elif dropdown_rule_btn.value=='Regla 6':
        df1=df[df['REGLA']=='Regla_6']
      elif dropdown_rule_btn.value=='Regla 7':
        df1=df[df['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 7':
        df1=df[df['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 8':
        df1=df[df['REGLA']=='Regla_8']
      elif dropdown_rule_btn.value=='Regla 9':
        df1=df[df['REGLA']=='Regla_9']
      else:
        df1=df
      return display(df1)

def create_error_tab_panel():

  dropdown_rule_btn=widgets.Dropdown(
    placeholder='Choose Someone',
    options=['Todos','Regla 1', 'Regla 2', 'Regla 3','Regla 4','Regla 5','Regla 6','Regla 7','Regla 8','Regla 9',],
    description='Filtro:',
    ensure_option=True,
    disabled=False)
  
  filter_rule_btn = widgets.Button(
    description='Filtrar',
    disabled=False,
    button_style='success',
    icon='check')
  
  download_error_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
    )


  def filter_error_rules(a):
    global df1
    with df_errores:
      clear_output()
      if dropdown_rule_btn.value=='Regla 1':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_1']
      elif dropdown_rule_btn.value=='Regla 2':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_2']
      elif dropdown_rule_btn.value=='Regla 3':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_3']
      elif dropdown_rule_btn.value=='Regla 4':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_4']
      elif dropdown_rule_btn.value=='Regla 5':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_5']
      elif dropdown_rule_btn.value=='Regla 6':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_6']
      elif dropdown_rule_btn.value=='Regla 7':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 7':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 8':
        df1= model['df_reglas'] [model['df_reglas'] ['REGLA']=='Regla_8']
      elif dropdown_rule_btn.value=='Regla 9':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_9']
      else:
        df1= model['df_reglas']
      return display(df1[['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION','REGLA']])
  
  def download_error_file(a):
    #print("download df.xlsx", df.shape)
    df1.iloc[:,0:19].to_excel("df.xlsx")

  #error_buttons_hbox = create_error_buttons()
  #dropdown_rule_btn = error_buttons_hbox.children[0]
  #filter_rule_btn = error_buttons_hbox.children[1]
  #process_file_btn.on_click(functools.partial(on_process_file_clicked,
  #                                            rule_chk_list=rule_checkboxes,
  #                                            pie_out=pie_output))

  filter_rule_btn.on_click(filter_error_rules)
  download_error_btn.on_click(download_error_file)

  calidad_items = [df_errores]
  calidad_column = widgets.VBox(calidad_items)
  

  return widgets.VBox(
    children=[widgets.Label("Errores destacados"),
        widgets.HBox([dropdown_rule_btn,filter_rule_btn,download_error_btn]),calidad_column
    ]
  )

def create_carga_page():
  # TODO añadir la pestaña error
  #children = [data_tab, error_tab]
  children = [create_carga_tab_panel(),create_error_tab_panel()]
  
  carga_tab = widgets.Tab()
  carga_tab.children = children
  carga_tab.set_title(0, 'Data')
  carga_tab.set_title(1, 'Errores')
  
  return VBox([Label(value="Carga de Datos"), 
               carga_tab])


In [71]:
######################
# Correlacion Page
######################

def on_correlacion_dropdown_x_changed(change, dropdown_y, corr_output):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return
  #model['conserje_diario']=change['new']
  num_x = model["num_columns"].index(change.new)
  x_idx=num_columns[num_x]
  num_y = model["num_columns"].index(dropdown_y.value)
  y_idx=num_columns[num_y]
  show_correlacion_graph(corr_output, x_idx, y_idx)


def on_correlacion_dropdown_y_changed(change, dropdown_x, corr_output):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return

  num_x = model["num_columns"].index(dropdown_x.value)
  x_idx =num_columns[num_x]
  num_y = model["num_columns"].index(change.new)
  y_idx = num_columns[num_y]
  show_correlacion_graph(corr_output, x_idx, y_idx)

def show_correlacion_graph(corr_output, x_idx, y_idx):

  global model
  df = model['df_final']
  with corr_output:
    clear_output()
    plt.scatter(df[x_idx], df[y_idx])
    plt.title('SCATTER')
    plt.xlabel(x_idx)
    plt.ylabel(y_idx);
    plt.show()



def create_correlacion_tab_panel():
  # Creación de widgets.Dropdown() para el scatterplot
  correlacion_x_dropdown=widgets.Dropdown(
    #placeholder='Choose Someone',
    options=num_columns,
    value=num_columns[0],
    description='Variable 1:'
    )
    #ensure_option=True,
    #disabled=False)
  correlacion_y_dropdown=widgets.Dropdown(
    #placeholder='Choose Someone',
    options=num_columns,
    value=num_columns[1],
    description='Variable 2:'
    )
    #ensure_option=True,
    #disabled=False)
  
  correlacion_output=widgets.Output()
  
  correlacion_x_dropdown.observe(functools.partial(on_correlacion_dropdown_x_changed,
                                              dropdown_y=correlacion_y_dropdown,
                                              corr_output=correlacion_output
                                              ))

  correlacion_y_dropdown.observe(functools.partial(on_correlacion_dropdown_y_changed,
                                              dropdown_x=correlacion_x_dropdown,
                                              corr_output=correlacion_output
                                              ))
  # default preview
  show_correlacion_graph(correlacion_output, num_columns[0],num_columns[1])
                     #model["num_columns"].index(num_columns[0]), 
                     #model["num_columns"].index(num_columns[1]))
  
  return widgets.VBox(
    children=[widgets.HBox([correlacion_x_dropdown, 
                      correlacion_y_dropdown]
                          ),
                      correlacion_output
              ]
            )

def create_correlacion_page():
  children1= [create_correlacion_tab_panel()]
  correlacion_tab = widgets.Tab()
  correlacion_tab.children = children1
  correlacion_tab.set_title(0, 'Correlación')
  
  return VBox([Label(value="Correlación de datos"), correlacion_tab])

In [72]:
################
# Outlier Page
################

def on_outlier_dropdown_city_changed(change, category_dropdown, type_dropdown, currency_dropdown, outlier_output):
  global model
  if change['name'] != 'value':
    return

  city = change.new
  show_outlier_graph(outlier_output, 3, city, category_dropdown.value, type_dropdown.value, currency_dropdown.value)


def on_outlier_dropdown_category_changed(change, city_dropdown, type_dropdown, currency_dropdown, outlier_output):
  global model
  if change['name'] != 'value':
    return

  category = change.new
  show_outlier_graph(outlier_output, 3, city_dropdown.value, category, type_dropdown.value, currency_dropdown.value)


def on_outlier_dropdown_type_changed(change, city_dropdown, category_dropdown, currency_dropdown, outlier_output):
  global model
  if change['name'] != 'value':
    return

  rent_type = change.new
  show_outlier_graph(outlier_output, 3, city_dropdown.value, category_dropdown.value, rent_type, currency_dropdown.value)


def on_outlier_dropdown_currency_changed(change, city_dropdown, category_dropdown, type_dropdown, outlier_output):
  global model
  if change['name'] != 'value':
    return

  currency = change.new
  show_outlier_graph(outlier_output, 3, city_dropdown.value, category_dropdown.value, type_dropdown.value, currency)


def filter_data(city, category, rent_type, currency):
  global model

  print('Generando gráficos con city=', city, ' category=', category, ' type=', rent_type, ' currency=', currency, ' ...')
  df = model['df_final']
  if category == 'TODAS' and rent_type == 'TODOS':
    df_filtered = df[(df['CIUDAD'] == city) & (df['MONEDA'] == currency)]  
  elif category == 'TODAS' and rent_type != 'TODOS':
    df_filtered = df[(df['CIUDAD'] == city) & (df['MONEDA'] == currency) & (df['TIPO'] == rent_type)]
  elif category != 'TODAS' and rent_type == 'TODOS':
    df_filtered = df[(df['CIUDAD'] == city) & (df['MONEDA'] == currency) & (df['CATEGORIA'] == category)]
  elif category != 'TODAS' and rent_type != 'TODOS':
    df_filtered = df[(df['CIUDAD'] == city) & (df['MONEDA'] == currency) & (df['CATEGORIA'] == category) & (df['TIPO'] == rent_type)]
  
  df_filtered.reset_index(drop=True, inplace=True)

  sc = StandardScaler()
  num_attribs_normalized = sc.fit_transform(df_filtered[num_columns])

  df_cat = df_filtered.drop(num_columns, axis=1)
  df_num = pd.DataFrame(num_attribs_normalized, columns=num_columns)
  return pd.concat([df_cat, df_num], axis=1)


def show_outlier_graph(outlier_output, num_components, city, category, rent_type, currency):

  df = filter_data(city, category, rent_type, currency)

  # Initialize figure with subplots
  fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.5, 0.5],
    specs=[[{"type": "scene"}, {"type": "scene"}]]
  )

  # Principal components
  pca_model = PCA(n_components = num_components)
  pca_data = pca_model.fit_transform(df[num_columns])
  variance_ratio = pca_model.explained_variance_ratio_.sum()


  pca_fig_data = px.scatter_3d(x=pca_data[:, 0], y=pca_data[:, 1], z=pca_data[:, 2], 
                              color=df['CATEGORIA'].values, hover_name=df.index)["data"]
  for i in range(0, len(pca_fig_data)):
    fig.add_trace(pca_fig_data[i], row=1, col=1)


  # T-SNE plot for train dataset
  sne_model = TSNE(n_components=num_components, random_state=42, perplexity=30)
  tsne_data = sne_model.fit_transform(df[num_columns])
  sne_fig_data = px.scatter_3d(x=tsne_data[:, 0], y=tsne_data[:, 1], z=tsne_data[:, 2], 
                              color=df['CATEGORIA'].values, hover_name=df.index)["data"]

  for i in range(0, len(sne_fig_data)):
    fig.add_trace(sne_fig_data[i], row=1, col=2)


  fig.update_xaxes(title_text="PCA", row=1, col=1)
  fig.update_yaxes(title_text="T-SNE", row=1, col=1)

  fig.update_layout(
    annotations=[
        dict(
            text=f'Explained Variance: {variance_ratio}\n Registros: {len(pca_data)} ',
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ], 
    title_text="Side By Side Subplots"
  )

  with outlier_output:
    clear_output()
    fig.show()


######## MAHALANOBIS ##################

def conf_slider_changed(change, df_mhlbs_out):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return
  #model['conserje_diario']=change['new']
  #num_x = model["num_columns"].index(change.new)
  #var_idx=num_columns[num_x]
  #num_y = model["num_columns"].index(dropdown_y.value)
  #y_idx=num_columns[num_y]
  ciudad=model['ciudad']
  moneda=model['moneda']
  conf_val=conf_sl.value
  show_df_mhlbs_graph(df_mhlbs_out,ciudad,moneda,conf_val)
  
def show_df_mhlbs_graph(df_mhlbs_out,ciudad,moneda,conf_val):
  global model,df1_final
  df = df1_final#model['df_final']
  with df_mhlbs_out:
    clear_output()
    criterios1=df['CIUDAD']==ciudad
    criterio2=df['MONEDA']==moneda
    criterio3=df['TIPO']=='ALQUILER'
    criterios=criterios1 & criterio2 & criterio3
    df_criterios=df[criterios]
    df_inputmhlbs=df_criterios[['DIAS ALQUILER','COSTO ALQUILER DIARIO','TOTAL ALQUILER','DIAS CONSERJE','COSTO CONSERJE DIARIO','TOTAL CONSERJE','COSTOS ADICIONALES SOLES','LIQUIDACION EN SOLES','LIQUIDACION EN DOLARES']]

    df_mhlbs=df_inputmhlbs.to_numpy()

    try:

      # Covariance matrix
      covariance  = np.cov(df_mhlbs , rowvar=False)

      # Covariance matrix power of -1
      #covariance_pm1 = np.linalg.matrix_power(covariance, -1)
      covariance_pm1 = np.linalg.pinv(covariance)
      # Center point
      centerpoint = np.mean(df_mhlbs , axis=0)

      # Distances between center point and 
      distances = []
      for i, val in enumerate(df_mhlbs):
            p1 = val
            p2 = centerpoint
            distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
            distances.append(distance)
      distances = np.array(distances)

      # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
      cutoff = chi2.ppf(conf_val, df_mhlbs.shape[1])

      df_outliers = pd.DataFrame(df_mhlbs[ distances > cutoff , :], columns = ['DIAS ALQUILER','COSTO ALQUILER DIARIO','TOTAL ALQUILER','DIAS CONSERJE','COSTO CONSERJE DIARIO','TOTAL CONSERJE','COSTOS ADICIONALES SOLES','LIQUIDACION EN SOLES','LIQUIDACION EN DOLARES'])

      display(df_outliers)
    
    except Exception:
      print("*********La data no contiene valores adecuados para estimar outliers*******")
      

    

def create_mhlbs_multivariado_tab_panel():
  
  ciudades_list=['LIMA','CUZCO','TRUJILLO','LORETO','AREQUIPA','CAJAMARCA','TACNA']
  ciudad_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=ciudades_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CIUDAD:',
    ensure_option=True,
    disabled=False)
  
  def ciudad_change(change,df_mhlbs_out,conf_sl):
    global model
    model['ciudad']=change['new']
    ciudad=model['ciudad']
    moneda=model['moneda']
    conf_val=conf_sl.value
    #show_boxplot_graph(boxplot_out,ciudad,moneda)
    show_df_mhlbs_graph(df_mhlbs_out,ciudad,moneda,conf_val)
  
  moneda_list=['PEN','USD','PEN / USD']
  moneda_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=moneda_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='MONEDA:',
    ensure_option=True,
    disabled=False)
  
  def moneda_change(change,df_mhlbs_out,conf_sl):
    global model
    model['moneda']=change['new']
    moneda=model['moneda']
    ciudad=model['ciudad']
    conf_val=conf_sl.value
    #show_boxplot_graph(boxplot_out,ciudad,moneda)
    show_df_mhlbs_graph(df_mhlbs_out,ciudad,moneda,conf_val)
  
  
  conf_slider=widgets.FloatSlider(
    value=0.95,
    min=0.80,
    max=0.98,
    step=0.01,
    description='CONFIANZA:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
  )

  def conf_change(change,df_mhlbs_out,conf_sl):
    global model
    conf_sl.value=change['new']
    conf_val=conf_sl.value
    moneda=model['moneda']
    ciudad=model['ciudad']
    #show_boxplot_graph(boxplot_out,ciudad,moneda)
    show_df_mhlbs_graph(df_mhlbs_out,ciudad,moneda,conf_val)

  #boxplot_output = widgets.Output()
  df_mhlbs_outlier= widgets.Output()

  moneda_drop.observe(functools.partial(moneda_change,
                                        #boxplot_out=boxplot_output,
                                        df_mhlbs_out=df_mhlbs_outlier,
                                        conf_sl=conf_slider
                                              ), names='value')

  ciudad_drop.observe(functools.partial(ciudad_change,
                                        #boxplot_out=boxplot_output,
                                        df_mhlbs_out=df_mhlbs_outlier,
                                        conf_sl=conf_slider
                                              ), names='value')

  conf_slider.observe(functools.partial(conf_change,
                                        #boxplot_out=boxplot_output,
                                        df_mhlbs_out=df_mhlbs_outlier,
                                        conf_sl=conf_slider
                                              ), names='value')


  #boxplot_univariado_buttons_Hbox=create_boxplot_univariado_buttons()
  return widgets.VBox(children=[HBox([ciudad_drop,moneda_drop,conf_slider]),
                                HBox([df_mhlbs_outlier])])

#######################

########
def var_num_changed(change, boxplot_out,df_box_out):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return
  #model['conserje_diario']=change['new']
  num_x = model["num_columns"].index(change.new)
  var_idx=num_columns[num_x]
  #num_y = model["num_columns"].index(dropdown_y.value)
  #y_idx=num_columns[num_y]
  ciudad=model['ciudad']
  moneda=model['moneda']
  show_boxplot_graph(boxplot_out,var_idx,ciudad,moneda)
  show_df_outlier_graph(df_box_out,var_idx,ciudad,moneda)

  
def show_boxplot_graph(boxplot_out,var_idx,ciudad,moneda):
  global model,df1_final
  df = df1_final#model['df_final']
  with boxplot_out:
    clear_output()
    criterios1=df['CIUDAD']==ciudad
    criterio2=df['MONEDA']==moneda
    criterio3=df['TIPO']=='ALQUILER'
    criterios=criterios1 & criterio2 & criterio3
    df_criterios=df[criterios]
    fig = px.box(df_criterios, y=var_idx,width=400, height=500)
    fig . update_traces ( boxpoints='suspectedoutliers')
    fig.show()

def show_df_outlier_graph(df_box_out,var_idx,ciudad,moneda):
  global model,df1_final
  df = df1_final#model['df_final']
  with df_box_out:
    clear_output()
    criterios1=df['CIUDAD']==ciudad
    criterio2=df['MONEDA']==moneda
    criterio3=df['TIPO']=='ALQUILER'
    criterios=criterios1 & criterio2 & criterio3
    df_criterios=df[criterios]
    quantil_75=df_criterios[var_idx].quantile(0.75)
    quantil_25=df_criterios[var_idx].quantile(0.25)
    IQR= quantil_75-quantil_25
    limite_inferior=np.float64(quantil_25 - (1.5*IQR))
    limite_superior=np.float64(quantil_75 + (1.5*IQR))
    cond1= df_criterios[var_idx]>limite_superior
    cond2= df_criterios[var_idx]<limite_inferior
    outlier=cond1 | cond2
    display(df_criterios[outlier][['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER','COSTO ALQUILER DIARIO', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES']])

def create_boxplot_univariado_tab_panel():
  
  ciudades_list=['LIMA','CUZCO','TRUJILLO','LORETO','AREQUIPA','CAJAMARCA','TACNA']
  ciudad_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=ciudades_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CIUDAD:',
    ensure_option=True,
    disabled=False)
  
  def ciudad_change(change,boxplot_out,var_drop,df_box_out):
    global model
    model['ciudad']=change['new']
    ciudad=model['ciudad']
    moneda=model['moneda']
    num_y = model["num_columns"].index(var_drop.value)
    var_idx=num_columns[num_y]
    show_boxplot_graph(boxplot_out,var_idx,ciudad,moneda)
    show_df_outlier_graph(df_box_out,var_idx,ciudad,moneda)
  
  moneda_list=['PEN','USD','PEN / USD']
  moneda_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=moneda_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='MONEDA:',
    ensure_option=True,
    disabled=False)
  
  def moneda_change(change,boxplot_out,var_drop,df_box_out):
    global model
    model['moneda']=change['new']
    moneda=model['moneda']
    ciudad=model['ciudad']
    num_y = model["num_columns"].index(var_drop.value)
    var_idx=num_columns[num_y]
    show_boxplot_graph(boxplot_out,var_idx,ciudad,moneda)
    show_df_outlier_graph(df_box_out,var_idx,ciudad,moneda)

  variable_num_dropdown=widgets.Dropdown(
    #placeholder='Choose Someone',
    options=num_columns,
    value=num_columns[0],
    description='Variable:'
    )
  
  boxplot_output = widgets.Output()
  df_boxplot_outlier= widgets.Output()

  moneda_drop.observe(functools.partial(moneda_change,
                                        boxplot_out=boxplot_output,
                                        var_drop=variable_num_dropdown,
                                        df_box_out=df_boxplot_outlier
                                              ), names='value')

  ciudad_drop.observe(functools.partial(ciudad_change,
                                        boxplot_out=boxplot_output,
                                        var_drop=variable_num_dropdown,
                                        df_box_out=df_boxplot_outlier
                                              ), names='value')

  variable_num_dropdown.observe(functools.partial(var_num_changed,
                                              boxplot_out=boxplot_output,
                                              df_box_out=df_boxplot_outlier
                                              ))

  #boxplot_univariado_buttons_Hbox=create_boxplot_univariado_buttons()
  return widgets.VBox(children=[HBox([ciudad_drop,moneda_drop,variable_num_dropdown]),
                                HBox([boxplot_output,df_boxplot_outlier])])

########

def create_pca_outlier_tab_panel():
  global model

  df = model['df']
  outlier_graph_city_dropdown = widgets.Dropdown(
    options=df['CIUDAD'].unique().tolist(),
    description='Ciudad:',
    ensure_option=True,
  )
  
  outlier_graph_category_dropdown = widgets.Dropdown(
    options=['TODAS'] + df['CATEGORIA'].unique().tolist(),
    description='Categoría:',
  )

  outlier_graph_type_dropdown = widgets.Dropdown(
    options=['TODOS'] + df['TIPO'].unique().tolist(),
    description='Tipos:',
  )

  outlier_graph_currency_dropdown = widgets.Dropdown(
    placeholder='Elije una moneda',
    options=df['MONEDA'].unique().tolist(),
    description='Moneda:',
  )

  outlier_scatter_output = widgets.Output()

  outlier_graph_city_dropdown.observe(functools.partial(on_outlier_dropdown_city_changed,
                                              category_dropdown=outlier_graph_category_dropdown,
                                              type_dropdown=outlier_graph_type_dropdown,
                                              currency_dropdown=outlier_graph_currency_dropdown,
                                              outlier_output=outlier_scatter_output))

  outlier_graph_category_dropdown.observe(functools.partial(on_outlier_dropdown_category_changed,
                                              city_dropdown=outlier_graph_city_dropdown,
                                              type_dropdown=outlier_graph_type_dropdown,
                                              currency_dropdown=outlier_graph_currency_dropdown,
                                              outlier_output=outlier_scatter_output))

  outlier_graph_type_dropdown.observe(functools.partial(on_outlier_dropdown_type_changed,
                                              city_dropdown=outlier_graph_city_dropdown,
                                              category_dropdown=outlier_graph_category_dropdown,
                                              currency_dropdown=outlier_graph_currency_dropdown,
                                              outlier_output=outlier_scatter_output))

  outlier_graph_currency_dropdown.observe(functools.partial(on_outlier_dropdown_currency_changed,
                                              city_dropdown=outlier_graph_city_dropdown,
                                              category_dropdown=outlier_graph_category_dropdown,
                                              type_dropdown=outlier_graph_type_dropdown,
                                              outlier_output=outlier_scatter_output))
  # default preview
  # show_outlier_graph(outlier_scatter_output, 
  #                    model["num_columns"].index(num_columns[0]), 
  #                    model["num_columns"].index(num_columns[1]))

  return widgets.VBox(
    children=[
        widgets.HBox(
            children=[Label(value="Parameters"),
                      outlier_graph_city_dropdown, 
                      outlier_graph_category_dropdown,
                      outlier_graph_type_dropdown,
                      outlier_graph_currency_dropdown]
          ),
        outlier_scatter_output
      ]
    )


def create_smart_grid_tab_panel():
  global model

  outlier_grid_output = widgets.Output()
  df = model['df']

  with outlier_grid_output:
    clear_output()
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

  return widgets.VBox(
    children=[
        widgets.HBox(
            children=[Label(value="Data"),]
          ),
        outlier_grid_output
      ]
    )


def create_outlier_page():
  children = [create_pca_outlier_tab_panel(), create_boxplot_univariado_tab_panel(), create_smart_grid_tab_panel(),create_mhlbs_multivariado_tab_panel() ]
  
  outlier_tab = widgets.Tab()
  outlier_tab.children = children
  outlier_tab.set_title(0, 'Outliers')
  outlier_tab.set_title(1, 'Boxplot_Univarido')
  outlier_tab.set_title(2, 'Grid')
  outlier_tab.set_title(3, 'Mahalanobis')
  
  
  return VBox([Label(value="Análisis de Outliers"), 
               outlier_tab])
  

In [73]:
################
# Register Page
################

def create_register_buttons():
  
  registrar_btn = widgets.Button(
    description='Registrar',
    disabled=False,
    button_style='success',
    icon='check')
  
  ciudades_list=['LIMA','CUZCO','TRUJILLO','LORETO','AREQUIPA','CAJAMARCA','TACNA']
  ciudad_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=ciudades_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CIUDAD:',
    ensure_option=True,
    disabled=False)
  def ciudad_change(change):
    global model
    model['ciudad']=change['new']
  ciudad_drop.observe(ciudad_change, names='value')

  tipo_list=['ALQUILER','USO INTERNO','CORTESIA']
  tipo_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=tipo_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='TIPO:',
    ensure_option=True,
    disabled=False)
  def tipo_change(change):
    global model
    model['tipo']=change['new']
  tipo_drop.observe(tipo_change, names='value')

  categoria_list=['DEPARTAMENTO','OFICINA','EVENTOS','LUJOS','STORAGE','CONSERJE']
  categoria_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=categoria_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CATEGORIA:',
    ensure_option=True,
    disabled=False)
  def categoria_change(change):
    global model
    model['categoria']=change['new']
  categoria_drop.observe(categoria_change, names='value')

  file_dia_in_text = widgets.Text(
    #value='',
    placeholder='YYYY-MM-DD',
    description= 'DIA (IN):',
    disabled=False)
  def dia_in_change(change):
    global model
    model['dia_in']=change['new']
  file_dia_in_text.observe(dia_in_change, names='value')

  file_hora_in_text = widgets.Text(
    #value='',
    placeholder='HH:MM:SS',
    description= 'HORA (IN):',
    disabled=False)
  def hora_in_change(change):
    global model
    model['hora_in']=change['new']
  file_hora_in_text.observe(hora_in_change, names='value')

  file_dia_out_text = widgets.Text(
    #value='',
    placeholder='YYYY-MM-DD',
    description= 'DIA (OUT):',
    disabled=False)
  def dia_out_change(change):
    global model
    model['dia_out']=change['new']
  file_dia_out_text.observe(dia_out_change, names='value')

  file_hora_out_text = widgets.Text(
    #value='',
    placeholder='HH:MM:SS',
    description= 'HORA (OUT):',
    disabled=False)
  def hora_out_change(change):
    global model
    model['hora_out']=change['new']
  file_hora_out_text.observe(hora_out_change, names='value')

  moneda_list=['PEN','USD','PEN / USD']
  moneda_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=moneda_list,  #la variable options se carga al presionar boton procesar
    value=None,
    #description='MONEDA:',
    ensure_option=True,
    disabled=False)
  def moneda_change(change):
    global model
    model['moneda']=change['new']
  moneda_drop.observe(moneda_change, names='value')

  file_alq_diario_text = widgets.Text(
    #value='',
    placeholder='COSTO ALQUILER DIARIO',
    #description= 'ALQ. DIARIO:',
    disabled=False)
  def alq_diario_change(change):
    global model
    model['alquiler_diario']=change['new']
  file_alq_diario_text.observe(alq_diario_change, names='value')

  file_dias_cons_text = widgets.Text(
    #value='',
    placeholder='NUM DÍAS CONSERJE',
    #description= 'DÍAS:',
    disabled=False)
  def dias_cons_change(change):
    global model
    model['dias_conserje']=change['new']
  file_dias_cons_text.observe(dias_cons_change, names='value')


  file_cons_diario_text = widgets.Text(
    #value='',
    placeholder='COSTO CONSERJE DIARIO',
    #description= 'CONS DIARIO:',
    disabled=False)
  def cons_diario_change(change):
    global model
    model['conserje_diario']=change['new']
  file_cons_diario_text.observe(cons_diario_change, names='value')

  file_cost_adicionales_text = widgets.Text(
    #value='',
    placeholder='COSTOS ADICIONALES SOLES',
    #description= 'ADICIONAL:',
    disabled=False)
  def adicionales_soles_change(change):
    global model
    model['adicionales_soles']=change['new']
  file_cost_adicionales_text.observe(adicionales_soles_change, names='value')

  file_agente_text = widgets.Text(
    #value='',
    placeholder='CODIGO AGENTE',
    #description= 'AGENTE:',
    disabled=False)
  def agente_change(change):
    global model
    model['agente']=change['new']
  file_agente_text.observe(agente_change, names='value')


  file_est_transaccion_text = widgets.Text(
    #value='',
    placeholder='ESTADO DE TRANSACCIÓN',
    #description= 'ESTADO:',
    disabled=False)
  def transaccion_change(change):
    global model
    model['estado_transaccion']=change['new']
  file_est_transaccion_text.observe(transaccion_change, names='value')
  
  ### \xa0 ES ESPACIO
  
  return widgets.VBox(children=[HBox([ciudad_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0MONEDA:'),moneda_drop]),
               HBox([tipo_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ALQUILER DIARIO:'),file_alq_diario_text]),
               HBox([categoria_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0DÍAS CONSERJE:'),file_dias_cons_text]),
               HBox([file_dia_in_text,widgets.Label('COSTO CONSERJE DIARIO\xa0:'),file_cons_diario_text]),
               HBox([file_hora_in_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ADICIONALES SOLES:'),file_cost_adicionales_text]),
               HBox([file_dia_out_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0CÓDIGO AGENTE:'),file_agente_text]),
               HBox([file_hora_out_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0ESTADO TRANSACCIÓN:'),file_est_transaccion_text]),
               HBox([registrar_btn])
                ])

def create_df_register_buttons():
  descargar_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success',
    icon='check')
  ver_datos_btn = widgets.Button(
    description='Ver',
    disabled=False,
    button_style='success',
    icon='check')
  return widgets.HBox([descargar_btn,ver_datos_btn])

def show_df_register(df_output):
  global df1_final
  #df1=registrar_fila
  with df_output:
    clear_output()
    display(df1_final[['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']])

def on_ver_datos_file_clicked(b,df_final):
  #df_final_output=widgets.Output()
  show_df_register(df_final)

def download_file(d):
    #print("download df.xlsx", df.shape)
    df1_final.to_excel("df.xlsx")

def create_df_register_tab_panel():
  df_register_buttons_Vbox=create_df_register_buttons()
  df_final_output=widgets.Output()
  ver_datos_file_btn=df_register_buttons_Vbox.children[1]
  ver_datos_file_btn.on_click(functools.partial(on_ver_datos_file_clicked,
                                              df_final=df_final_output))
  descargar_datos_file_btn=df_register_buttons_Vbox.children[0]
  descargar_datos_file_btn.on_click(download_file)
  return widgets.VBox([df_register_buttons_Vbox,df_final_output])


def create_register_tab_panel():
  register_buttons_hbox=create_register_buttons()
  registrar_file_btn = register_buttons_hbox.children[7].children[0]
  registrar_file_btn.on_click(registrar_fila)
  return register_buttons_hbox

def create_register_page():
  children2= [create_register_tab_panel(),create_df_register_tab_panel()]
  registrar_tab = widgets.Tab()
  registrar_tab.children = children2
  registrar_tab.set_title(0, 'Registrar')
  registrar_tab.set_title(1, 'Datos')

  return VBox([Label(value="Registrar datos"), registrar_tab])



###########################
# Funciones para registrar
###########################

def indicar_iata():
  global model
  
  ciudad=model['ciudad']
  if ciudad=='LIMA':
    a='LIM'
  elif ciudad=='CUZCO':
    a='CUZ'
  elif ciudad=='TRUJILLO':
    a='TRU'
  elif ciudad=='LORETO':
    a='LTO'
  elif ciudad=='AREQUIPA':
    a='AQP'
  elif ciudad=='CAJAMARCA':
    a='CJA'
  elif ciudad=='TACNA':
    a='TCQ'
  return a

def convertir_fecha(a,b):
  fecha=str(a)+" "+str(b)
  fecha=pd.Timestamp(fecha)
  return fecha

def calcular_dias(inicio,final):
  dias_alq=np.int64((final - inicio).ceil('d').days)
  return dias_alq

def tipo_moneda(alquiler,conserje,adicionales):
  global model
  if model['moneda']=='PEN':
    liquidacion_soles= alquiler + conserje + adicionales
    liquidacion_dolares= 0.0
  elif model['moneda']=='USD':
    liquidacion_soles= 0.0
    liquidacion_dolares= alquiler + conserje
  elif model['moneda']=='PEN / USD':
    liquidacion_soles= adicionales
    liquidacion_dolares= alquiler + conserje
  return liquidacion_soles,liquidacion_dolares

def registrar_fila(a):
  global model,df1_final
  
  ciudad=model['ciudad']
  iata=indicar_iata()
  tipo=model['tipo']
  categoria=model['categoria']
  file_dia_in_text=model['dia_in']
  file_hora_in_text=model['hora_in']
  file_dia_out_text=model['dia_out']
  file_hora_out_text=model['hora_out']
  check_in=convertir_fecha(file_dia_in_text,file_hora_in_text)
  check_out=convertir_fecha(file_dia_out_text,file_hora_out_text)
  moneda=model['moneda']
  dias_alquiler=calcular_dias(check_in,check_out)
  costo_alquiler=np.float64(model['alquiler_diario'])
  total_alquiler=np.round((calcular_dias(check_in,check_out))*(costo_alquiler),2)
  dias_conserje=np.int64(model['dias_conserje'])
  costo_conserje=np.float64(model['conserje_diario'])
  total_conserje=np.round((dias_conserje)*(costo_conserje),2)
  costos_adicionales_soles=np.float64(model['adicionales_soles'])
  costos_adicionales_dolares=''
  liquidacion_soles=tipo_moneda(total_alquiler,total_conserje,costos_adicionales_soles)[0]
  liquidacion_dolares=tipo_moneda(total_alquiler,total_conserje,costos_adicionales_soles)[1]
  agente=model['agente']
  estado_transaccion=model['estado_transaccion']
  data=np.array([[ciudad,iata,tipo,categoria,check_in,check_out,moneda,dias_alquiler,costo_alquiler,total_alquiler,dias_conserje,costo_conserje,total_conserje,costos_adicionales_soles,costos_adicionales_dolares,liquidacion_soles,liquidacion_dolares,agente,estado_transaccion]])
  pr=pd.DataFrame(data,columns=model['df_final'].columns)
  pr=pr.infer_objects() ### para que mantengan su tipo de dato
  pr['COSTOS ADICIONALES DOLARES']=pd.to_numeric(pr['COSTOS ADICIONALES DOLARES'])
  df1_final=pd.concat([df1_final,pr])
  df1_final.reset_index(drop=True, inplace=True)
  return df1_final



In [74]:
######################
# Fluctuaciones page
######################

def show_fluctuaciones_graph(fluctuaciones_out,ciudad,moneda,categoria):
  global model,df1_final
  df = df1_final
  with fluctuaciones_out:
    clear_output()
    df_fluctuacion=df1_final[df1_final['TIPO']=='ALQUILER']
    c1=df_fluctuacion['MONEDA']==moneda 
    c2=df_fluctuacion['CIUDAD']==ciudad
    c3=df_fluctuacion['CATEGORIA']==categoria
    condicion=c1&c2&c3
    #condicion=c1&c2
    data_area=df_fluctuacion[condicion]
    #data_area.reset_index(drop=True, inplace=True)
    pr1=data_area['CHECK IN'].dt.date
    if moneda=='PEN':
      pr2=data_area['LIQUIDACION EN SOLES']/data_area['DIAS ALQUILER']
    else:
      pr2=data_area['LIQUIDACION EN DOLARES']/data_area['DIAS ALQUILER']
    pr3=data_area['DIAS ALQUILER']
    frame = { 'CHECK IN': pr1, 'LIQUIDACION SOLES': pr2 ,'DIAS ALQUILER':pr3 }
    result = pd.DataFrame(frame)
    lista_day=result['CHECK IN'].tolist()
    lista_liq=[]
    lista_fecha=[]
    for i in range(0,result.shape[0]):
      fe=lista_day[i]
      for j in range(0,result['DIAS ALQUILER'].iloc[i]):
        fecha=fe+ datetime.timedelta(days=j)
        liquidacion=result['LIQUIDACION SOLES'].iloc[i]
        lista_fecha.append(fecha)
        lista_liq.append(liquidacion)
    df_ejemplo = pd.DataFrame(list(zip(lista_fecha,lista_liq)), columns = ['FECHA','LIQUIDACION'])
    df_ejemplo['FECHA']= pd.to_datetime(df_ejemplo['FECHA'], format="%Y/%m/%d")
    df_ejemplo=df_ejemplo.groupby('FECHA').sum()
    df_ejemplo_diario=df_ejemplo.resample('D').mean()
    df_ejemplo_diario=df_ejemplo_diario.fillna(0)
    fig1 = px.area(df_ejemplo_diario, x=df_ejemplo_diario.index, y=df_ejemplo_diario['LIQUIDACION'],width=1100, height=500)
    fig1.show()

def create_fluctuaciones_tab_panel():
  
  ciudades_list=['LIMA','CUZCO','TRUJILLO','LORETO','AREQUIPA','CAJAMARCA','TACNA']
  ciudad_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=ciudades_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CIUDAD:',
    ensure_option=True,
    disabled=False)
  
  def ciudad_change(change,fluctuaciones_out):
    global model
    model['ciudad']=change['new']
    ciudad=model['ciudad']
    moneda=model['moneda']
    categoria=model['categoria']
    show_fluctuaciones_graph(fluctuaciones_out,ciudad,moneda,categoria)
  
  moneda_list=['PEN','USD','PEN / USD']
  moneda_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=moneda_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='MONEDA:',
    ensure_option=True,
    disabled=False)
  
  def moneda_change(change,fluctuaciones_out):
    global model
    model['moneda']=change['new']
    ciudad=model['ciudad']
    moneda=model['moneda']
    categoria=model['categoria']
    show_fluctuaciones_graph(fluctuaciones_out,ciudad,moneda,categoria)
  
  categoria_list=['DEPARTAMENTO','OFICINA','EVENTOS','LUJOS','STORAGE','CONSERJE']
  categoria_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=categoria_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CATEGORIA:',
    ensure_option=True,
    disabled=False)
  
  def categoria_change(change,fluctuaciones_out):
    global model
    model['categoria']=change['new']
    ciudad=model['ciudad']
    moneda=model['moneda']
    categoria=model['categoria']
    show_fluctuaciones_graph(fluctuaciones_out,ciudad,moneda,categoria)
  
  grafico_fluctuaciones_output = widgets.Output()
  #df_boxplot_outlier= widgets.Output()

  moneda_drop.observe(functools.partial(moneda_change,
                                        fluctuaciones_out=grafico_fluctuaciones_output
                                              ), names='value')

  ciudad_drop.observe(functools.partial(ciudad_change,
                                        fluctuaciones_out=grafico_fluctuaciones_output
                                              ), names='value')

  categoria_drop.observe(functools.partial(categoria_change,
                                        fluctuaciones_out=grafico_fluctuaciones_output
                                              ), names='value')
  
  return widgets.VBox(children=[HBox([ciudad_drop,moneda_drop,categoria_drop]),
                                HBox([grafico_fluctuaciones_output])])

def create_fluctuaciones_page():
  children= [create_fluctuaciones_tab_panel()]
  fluctuaciones_tab = widgets.Tab()
  fluctuaciones_tab.children = children
  fluctuaciones_tab.set_title(0, 'Fluctuaciones')

  return VBox([Label(value="Fluctuaciones"), fluctuaciones_tab])

In [75]:
##############
# Main layout
##############

def create_logo_pucp_image():
  logo_url = "https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg"
  logo_image = IPython.display.Image(logo_url, 
                                     width=290)
  return widgets.Image(value=logo_image.data, 
                       format='jpg',
                       width=290,
                       height=145)


def create_main_title():
  title = "Detección de patrones de alquiler por ciudad"
  title_layout = Layout(width='60%', 
                        display='flex', 
                        flex_flow='column', 
                        align_items='center', 
                        justify_content='center')
  return widgets.Label(value=title, 
                       layout=title_layout)


def create_global_header():
  return HBox([create_main_title(), 
               create_logo_pucp_image()])


# Main output
def create_content_output():
  main_output = widgets.Output(layout={'border': '1px solid black'})
  model["main_output"] = main_output
  return main_output


#  Main Menu
def show_load_page(b):
  with model["main_output"]:
    clear_output()
    display(create_carga_page())

def show_correlation_page(b):
  with model["main_output"]:
    clear_output()
    display(create_correlacion_page())

def show_outliers_page(b):
  with model["main_output"]:
    clear_output()
    display(create_outlier_page())

def show_register_page(b):
  with model["main_output"]:
    clear_output()
    display(create_register_page())

def show_fluctuaciones_page(b):
  with model["main_output"]:
    clear_output()
    display(create_fluctuaciones_page())

def create_global_menu():
  items_layout = Layout(width='auto') # override the default width of the button to 'auto' to let the button grow
  main_menu_layout = Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='solid',
                            width='90%')
  
  menu_options = ['Carga', 'Correlación', 'Outliers','Registrar','Fluctuaciones']
  main_menu_buttons = [Button(description=menu_option, layout=items_layout, button_style='success') for menu_option in menu_options]

  main_menu_buttons[0].on_click(show_load_page)
  main_menu_buttons[1].on_click(show_correlation_page)
  main_menu_buttons[2].on_click(show_outliers_page)
  main_menu_buttons[3].on_click(show_register_page)
  main_menu_buttons[4].on_click(show_fluctuaciones_page)

  return Box(children=main_menu_buttons, 
             layout=main_menu_layout)

In [76]:
AppLayout(header=create_global_header(),
          left_sidebar=create_global_menu(),
          center=create_content_output(),
          pane_widths=[1, 5, 0])

AppLayout(children=(HBox(children=(Label(value='Detección de patrones de alquiler por ciudad', layout=Layout(a…

In [ ]:
# model['df_original'].head(5)